In [1]:
import pandas as pd

In [13]:
ROOT = "/home/lazye/Documents/ufrgs/mcs/clip/clip-bias-explore/\
fair-face-classification"
RESULTS_PATH = ROOT + "/data/results"

# original paper preds df with race_gender preds
opd_df = pd.read_csv(RESULTS_PATH+"/original_paper_preds.csv")

# top synm preds df
topk_df = pd.read_csv(RESULTS_PATH+"/top_k_synms.csv")

# avg sum synm preds df
avg_df = pd.read_csv(RESULTS_PATH+"/avg_sum_synms.csv") 


## Original CLIP Paper Predictions Analysis

In [14]:
opd_df.drop(columns=['service_test'], inplace=True)
print(opd_df.head(1))
print('\ntransforming gender_preds to binary classes...\n')
opd_df['gender_preds'] = opd_df['gender_preds'].map(lambda x: x.split('_')[-1])
print(opd_df.head(1))


        file  age gender        race          gender_preds
0  val/1.jpg  3-9   Male  East Asian  Southeast Asian_Male

transforming gender_preds to binary classes...

        file  age gender        race gender_preds
0  val/1.jpg  3-9   Male  East Asian         Male


In [16]:
print("Validation set info:\n")
print(opd_df.race.value_counts(
    normalize=True).mul(100).round(1).astype(str) + '%')


Validation set info:

race
White              19.0%
Latino_Hispanic    14.8%
Black              14.2%
East Asian         14.2%
Indian             13.8%
Southeast Asian    12.9%
Middle Eastern     11.0%
Name: proportion, dtype: object


In [26]:
race_count = opd_df.race.value_counts()
race_percent = opd_df.race.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
gender_count = opd_df.gender.value_counts()
gender_percent = opd_df.gender.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

val_info_race = {"race_count": race_count,
                 "race_percent": race_percent}

val_info_race_df = pd.DataFrame(val_info_race)
print("Race info")
print(val_info_race_df.head())

val_info_gender = {"gender_count": gender_count,
                   "gender_percent": gender_percent}

val_info_gender_df = pd.DataFrame(val_info_gender)
print("\nGender info")
print(val_info_gender_df.head())


Race info
                 race_count race_percent
race                                    
White                  2085        19.0%
Latino_Hispanic        1623        14.8%
Black                  1556        14.2%
East Asian             1550        14.2%
Indian                 1516        13.8%

Gender info
        gender_count gender_percent
gender                             
Male            5792          52.9%
Female          5162          47.1%
